# Libs

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from mealpy import FloatVar, ES

# Load Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

# Set functions

In [ ]:
def build_autoencoder_classifier(params):
    hidden_layers = int(round(params[0]))
    neurons = int(round(params[1]))
    lr = float(params[2])

    input_layer = Input(shape=(784,))
    x = input_layer
    for _ in range(hidden_layers):
        x = Dense(neurons, activation='relu')(x)
    latent = Dense(32, activation='relu')(x)

    # Decoder
    x = latent
    for _ in range(hidden_layers):
        x = Dense(neurons, activation='relu')(x)
    output_layer = Dense(784, activation='sigmoid')(x)

    autoencoder = Model(input_layer, output_layer)
    encoder = Model(input_layer, latent)
    autoencoder.compile(optimizer=Adam(lr), loss='mse')
    return autoencoder, encoder

def build_classifier(encoder):
    for layer in encoder.layers:
        layer.trainable = False
    input_latent = encoder.input
    x = encoder.output
    x = Dense(64, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(input_latent, output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def objective_function(solution):
    try:
        autoencoder, encoder = build_autoencoder_classifier(solution)
        autoencoder.fit(x_train, x_train, epochs=5, batch_size=256, verbose=0)

        classifier = build_classifier(encoder)
        classifier.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)
        preds = classifier.predict(x_test)
        acc = accuracy_score(y_test, np.argmax(preds, axis=1))
        return 1 - acc  # menor é melhor
    except Exception as e:
        print("Erro ao avaliar indivíduo:", solution)
        return 1.0  # penalização forte

# Set params for the problem

In [ ]:
problem_dict = {
    "bounds": FloatVar(
        lb=(1, 16, 0.0001),
        ub=(3, 256, 0.01),
        name="autoencoder_params"
    ),
    "minmax": "min",
    "obj_func": objective_function
}

# Run ES

In [ ]:
model = ES.OriginalES(epoch=10, pop_size=5, lamda=0.75, verbose=True)
g_best = model.solve(problem_dict)

# Results

In [ ]:
print("\nMelhor solução encontrada:")
print(f"Configuração ótima: Camadas={int(round(g_best.solution[0]))}, Neurônios={int(round(g_best.solution[1]))}, LR={g_best.solution[2]:.5f}")
print(f"Fitness: {g_best.target.fitness:.5f}")

plt.plot(model.history.list_global_best_fit)
plt.title("Convergência do Evolution Strategy (MEALpy)")
plt.xlabel("Geração")
plt.ylabel("Fitness (1 - acurácia)")
plt.grid(True)
plt.show()